In [ ]:
# Import the required packages and libraries
import json
import requests
import datetime
import pandas as pd
import os
from dotenv import load_dotenv 
from pathlib import Path
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from pyprediktormapclient.auth_client import AUTH_CLIENT
from pyprediktormapclient.analytics_helper import AnalyticsHelper
from pyprediktormapclient.shared import *

In [ ]:
# Consider obtaining the envrionment variables from .env file if you are running this locally from source.
dotenv_path = Path("../.env")
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
username = os.environ["USERNAME"]
password = os.environ["PASSWORD"]
opcua_rest_url = os.environ["OPC_UA_REST_URL"]
opcua_server_url = os.environ["OPC_UA_SERVER_URL"]
model_index_url = os.environ["MODEL_INDEX_URL"]
ory_url = os.environ["ORY_URL"]

In [ ]:
# Getting ory bearer token
auth_client = AUTH_CLIENT(rest_url=ory_url, username=username, password=password)
auth_client.request_new_ory_token()

In [ ]:
url = model_index_url
headers = {"Content-Type": "application/json",
            "Accept": "application/json"
            }
auth_client = auth_client
session: requests.Session = None

In [ ]:
if auth_client is not None:
    if auth_client.token is not None:
        headers["Authorization"] = f"Bearer {auth_client.token.session_token}"
    if hasattr(auth_client, 'session_token'):
        headers["Cookie"] = f"ory_kratos_session={auth_client.session_token}"

body = {"Connection": {"Url": url, "AuthenticationType": 1}}

In [ ]:
def get_object_types() -> str:
        content = request_from_api(url, "GET", "query/object-types", headers=headers, session=session)

        return content

In [ ]:
object_types = get_object_types()
object_types

In [ ]:
def get_namespace_array() -> str:
        """Get the namespace array

        Returns:
            str: the JSON returned from the server
        """
        content = request_from_api(url, "GET", "query/namespaces", headers=headers, session=session)

        return content

In [ ]:
name_space_array = get_namespace_array()
name_space_array

In [ ]:
def get_object_type_id_from_name(type_name: str) -> str:
        """Get object type id from type name

        Args:
            type_name (str): type name

        Returns:
            str: the type id that corresponds with the id (None if not found)
        """
        try:
            obj_type = next(
                item for item in object_types if item["BrowseName"]["Name"] == type_name
            )
        except StopIteration:
            obj_type = {}

        if obj_type:
            node_id = obj_type.get("NodeId", {})
            object_type_id = f'{node_id.get("Namespace")}:{node_id.get("IdType")}:{node_id.get("Id")}'
        else:
            object_type_id = None

        return object_type_id.split(':')

In [ ]:
object_type_id = get_object_type_id_from_name("SiteType")
object_type_id

In [ ]:
int(object_type_id[0])

In [ ]:
def get_objects_of_type(type_name: str) -> str:
        """Function to get all the types of an object

        Args:
            type_name (str): type name

        Returns:
            A json-formatted string with the objects (or None if the type is not found)
        """
        object_type_id = get_object_type_id_from_name(type_name)

        if object_type_id is None:
            return None
        
        namespace, id_type, id_ = object_type_id

        body = json.dumps({"Id": int(id_), "IdType": int(id_type), "Namespace": int(namespace)})
        content = request_from_api(url, "POST", "query/objects-of-type-with-variables-and-properties", body, headers=headers, session=session)

        return content

In [ ]:
objects_of_type = get_objects_of_type("SiteType")
objects_of_type

In [ ]:
def get_objects_of_type_with_vars_and_pros(
        type_name: str,

    ) -> str:

        # if type_name is None:
        #     raise ValidationError("type_name cannot be None or empty")
    
        type_id = get_object_type_id_from_name(type_name)

        body = json.dumps(
            {
                "FilterResultsByNamespaces": {
                    "IdType": 1,
                    "Values": name_space_array
                },
                "Id": type_id[2],  
                "Namespace": int(type_id[0]),
                "IdType": int(type_id[1]),
                "Page": 1,
                "PageSize": 1
            }
        )
        content = request_from_api(url, "POST", "query/objects-of-type-with-variables-and-properties", body, headers=headers, session=session)
        return content

{
  "FilterResultsByNamespaces": {
    "IdType": 1,
    "Values": [
      "http://scatecsolar.com/EG-AS",
      "http://scatecsolar.com/JO-GL"
    ]
  },
  "Id": 1009,
  "Namespace": 1,
  "IdType": 0,
  "Page": 1,
  "PageSize": 50
}

In [ ]:
objects_of_type_vars_pros = get_objects_of_type_with_vars_and_pros("SiteType")
objects_of_type_vars_pros

In [ ]:
def expand_variables(data):
    expanded_data = []

    for reference in data['BrowseResult']['References']:
        for variable in reference.get('Variables', []):
            # Extract variable details
            var_details = {
                'Variable_NodeId': variable['NodeId']['Id'],
                'Variable_BrowseName': variable['BrowseName']['Name'],
                'Variable_DisplayName': variable['DisplayName']['Text'],
                'Variable_NodeClass': variable['NodeClass'],
                'Variable_TypeDefinition_Id': variable['TypeDefinition']['Id'],
                'Variable_Value': variable['Value']
            }

            # Extract properties of the variable
            for prop in variable.get('Properties', []):
                prop_details = {
                    'Property_NodeId': prop['NodeId']['Id'],
                    'Property_BrowseName': prop['BrowseName']['Name'],
                    'Property_DisplayName': prop['DisplayName']['Text'],
                    'Property_NodeClass': prop['NodeClass'],
                    'Property_TypeDefinition_Id': prop['TypeDefinition']['Id'],
                    'Property_Value': prop['Value']
                }
                # Combine variable details with property details
                combined = {**var_details, **prop_details}
                expanded_data.append(combined)

    return expanded_data

In [ ]:
expanded_data = expand_variables(objects_of_type_vars_pros)
df = pd.DataFrame(expanded_data)
df

In [ ]:
df.columns

In [ ]:
def get_object_descendants(

        root_type_name: str,
        type_name: str,

    ) -> str:
        """Retrieve object descendants based on root and type names.

        Args:

            root_type_name (str): The type name of the root object.
            type_name (str): The type name of the descendant object.


        Returns:

            str: A JSON-formatted string with descendants data of the selected object (or None if the type is not found).
        """

        if root_type_name is None or not type_name:
            raise ValidationError("root_type_name and type_name cannot be None or empty")
    
        root_id = get_object_type_id_from_name(root_type_name)
        type_id = get_object_type_id_from_name(type_name)

        body = json.dumps(
            {
                "FilterResultsByNamespaces": {
                    "IdType": 1,
                    "Values": name_space_array
                },
                "ReferenceTypeIds": [
                    {
                        "Id": "4003",
                        "IdType": 0,
                        "Namespace": 1
                    },
                    {
                        "Id": 4004,
                        "IdType": 0,
                        "Namespace": 1
                    }
                ],
                "RootNodeId": {
                    "Id": root_id[2],  
                    "IdType": int(root_id[1]),
                    "Namespace": int(root_id[0])
                },
                "TypeNodeId": {

                    "Id": type_id[2],  
                    "IdType": int(type_id[1]),
                    "Namespace": int(type_id[0])

                }
            }

        )
        content = request_from_api(url, "POST", "query/object-descendants?include_members=true&any_level=true", body, headers=headers, session=session)


        return content

In [ ]:
object_descendants = get_object_descendants("StringCombinerType", "StringSetType")
object_descendants

In [ ]:
object_descendants = get_object_descendants("SiteType", "SubSsiteType")
object_descendants

In [ ]:
def flatten_json(data):
    flat_list = []

    def flatten(item, parent_key=''):
        if isinstance(item, dict):
            for key, value in item.items():
                new_key = f"{parent_key}{key}_" if parent_key else key
                flatten(value, new_key)
        elif isinstance(item, list):
            for i, value in enumerate(item):
                flatten(value, f"{parent_key}{i}_")
        else:
            flat_list.append((parent_key[:-1], item))

    flatten(data)
    return dict(flat_list)

In [ ]:
flattened_data = []
for reference in object_descendants['BrowseResult']['References']:
    flat_ref = flatten_json(reference)
    flattened_data.append(flat_ref)

In [ ]:
df = pd.DataFrame(flattened_data)
df

In [ ]:
def flatten_members(data):
    flat_list = []

    def flatten(item, parent_key=''):
        if isinstance(item, dict):
            for key, value in item.items():
                new_key = f"{parent_key}{key}_" if parent_key else key
                flatten(value, new_key)
        elif isinstance(item, list):
            for i, value in enumerate(item):
                flatten(value, f"{parent_key}{i}_")
        else:
            flat_list.append((parent_key[:-1], item))

    flatten(data)
    return dict(flat_list)

In [ ]:
all_flattened_data = []
for reference in object_descendants['BrowseResult']['References']:
    flat_ref = flatten_json(reference)
    if 'Members' in reference:
        for member in reference['Members']:
            flat_member = flatten_members(member)
            combined = {**flat_ref, **flat_member}
            all_flattened_data.append(combined)
    else:
        all_flattened_data.append(flat_ref)

In [ ]:
df_full = pd.DataFrame(all_flattened_data)
df_full